# Wind Field Generation from Trained DRD Model

In [ ]:
from drdmannturb.calibration import CalibrationProblem
from drdmannturb.data_generator import OnePointSpectraDataGenerator
from drdmannturb.shared.parameters import NNParameters, ProblemParameters, LossParameters, PhysicalParameters
from drdmannturb.interfaces.generate_wind import GenerateWind

import torch
import torch.nn as nn
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu" 

# v2: torch.set_default_device('cuda:0')
if torch.cuda.is_available():
    torch.set_default_tensor_type("torch.cuda.FloatTensor")

import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
Type_Model = 'NN' ### 'FPDE_RDT', 'Mann', 'VK', 'NN'
nBlocks    = 3

normalize = True
friction_velocity = 2.683479938442173
reference_height = 180.0
roughness_height = 0.75
grid_dimensions = np.array([1200.0, 864.0, 576.0])
grid_levels = np.array([5, 3, 5])
seed = None #9000

path_to_parameters = './results/EddyLifetimeType.CUSTOMMLP_DataType.KAIMAL.pkl'

##########################################
### Wind generation
##########################################
wind = GenerateWind(friction_velocity, reference_height, grid_dimensions, grid_levels, seed, model=Type_Model, path_to_parameters=path_to_parameters)
for _ in range(nBlocks):
    wind()
    wind_field = wind.total_wind

# TODO: these should be moved into a GenerateWind method...? 
##########################################
### Scaling of the field (normalization)
##########################################
if normalize == True:
# h = np.array(grid_dimensions/wind_field.shape[0:-1])
# h = np.array(1/wind_field.shape[0],1/wind_field.shape[1],1/wind_field.shape[2])
    sd = np.sqrt(np.mean(wind_field**2))
    wind_field = wind_field/sd
    wind_field *= 4.26 # rescale to match Mann model 

JCSS_law = lambda z, z_0, delta, u_ast: u_ast/0.41 * ( np.log(z/z_0+1.0) + 5.57*z/delta - 1.87*(z/delta)**2 - 1.33*(z/delta)**3 + 0.25*(z/delta)**4 )
log_law = lambda z, z_0, u_ast: u_ast * np.log(z/z_0+1.0)/0.41

z = np.linspace(0.0,grid_dimensions[2], 2**(grid_levels[2])+1)
# mean_profile_z = JCSS_law(z, roughness_height, 10.0, friction_velocity)
mean_profile_z = log_law(z, roughness_height, friction_velocity)

mean_profile = np.zeros_like(wind_field)
mean_profile[...,0] = np.tile(mean_profile_z.T, (mean_profile.shape[0], mean_profile.shape[1], 1))

# wind_field = mean_profile
wind_field += mean_profile

wind_field *= 40/63

In [ ]:
wind_field.shape

In [ ]:
spacing = tuple(grid_dimensions/(2.0**grid_levels + 1))

wind_field_vtk = tuple([np.copy(wind_field[...,i], order='C') for i in range(3)])

cellData = {'grid': np.zeros_like(wind_field[...,0]), 'wind': wind_field_vtk}
# TODO: finish writing out to VTK properly

In [ ]:
x = np.array([spacing[0]*n for n in range(wind_field.shape[0])])
y = np.array([spacing[1]*n for n in range(wind_field.shape[1])])
z = np.array([spacing[2]*n for n in range(wind_field.shape[2])])

X, Y, Z = np.meshgrid(x, y, z)

fig = make_subplots(
    rows = 1, cols = 3, 
    subplot_titles=("x Component", "y Component", "z Component"), 
    specs= [[{'type': 'volume'}, {'type': 'volume'}, {'type': 'volume'}]]
)

fig.add_trace(go.Volume(
        x = X.flatten(), 
        y = Y.flatten(), 
        z = Z.flatten(), 
        value = wind_field_vtk[0].flatten(),
        coloraxis='coloraxis',
        opacity=0.5
    ), row = 1, col = 1
)

fig.add_trace(go.Volume(
        x = X.flatten(), 
        y = Y.flatten(), 
        z = Z.flatten(), 
        value = wind_field_vtk[1].flatten(),
        coloraxis='coloraxis',
        opacity=0.5
    ), row = 1, col = 2
)

fig.add_trace(go.Volume(
        x = X.flatten(), 
        y = Y.flatten(), 
        z = Z.flatten(), 
        value = wind_field_vtk[2].flatten(),
        coloraxis='coloraxis',
        opacity=0.5
    ), row = 1, col = 3
)

fig.update_layout(scene = dict( aspectmode='data'), scene2 = dict( aspectmode='data'), scene3 = dict( aspectmode='data')) 

fig.update_layout(
    scene = dict(
        xaxis = dict(visible=False),
        yaxis = dict(visible=False),
        zaxis = dict(visible=False)
        ), 
    scene2 = dict(
        xaxis = dict(visible=False),
        yaxis = dict(visible=False),
        zaxis = dict(visible=False)
        ), 
    scene3 = dict(
        xaxis = dict(visible=False),
        yaxis = dict(visible=False),
        zaxis = dict(visible=False)
        )
    )

fig.update_coloraxes(colorscale='spectral_r')

fig.show()

In [ ]:
wind_magnitude = np.sqrt(wind_field_vtk[0]**2+wind_field_vtk[1]**2+wind_field_vtk[2]**2)


fig = go.Figure(
    data = go.Volume(
        x=X.flatten(), 
        y=Y.flatten(), 
        z=Z.flatten(), 
        value=wind_magnitude.flatten(), 
        opacity=0.5, 
        colorscale='spectral_r', 
        colorbar={"title": '|U(x)|'}
    ), 
)

fig.update_layout(scene = dict( aspectmode='data')) 

fig.update_layout(
    scene = dict(
        xaxis = dict(visible=False),
        yaxis = dict(visible=False),
        zaxis = dict(visible=False)
        ), 
)

fig.update_layout(title_text="Fluctuation Vector Magnitude", title_x=0.5)

fig.show()